# Exercise: Working with Satellite Data

## Aim: Use python tools to search for, download, and manipulate satellite data

### Issues covered:
- Search for and request data from a public STAC catalogue of satellite imagery
- Download satellite imagery as raster data 
- Read rasters into python using the rioxarray package
- Visualise single/multi-band raster data

### Introduction

A number of satellites take snapshots of the Earth’s surface from space. The images recorded by these remote sensors represent a very precious data source for any activity that involves monitoring changes on Earth. Satellite imagery is typically provided in the form of geospatial raster data, with the measurements in each grid cell (“pixel”) being associated to accurate geographic coordinate information.

In this notebook exercise we will explore how to access open satellite data using Python. In particular, we will consider [the Sentinel-2 data collection that is hosted on AWS](https://registry.opendata.aws/sentinel-2-l2a-cogs). This dataset consists of multi-band optical images acquired by the two satellites of [the Sentinel-2 mission](https://sentinel.esa.int/web/sentinel/missions/sentinel-2) and it is continuously updated with new images.


# 1. Search for satellite imagery

**The SpatioTemporal Asset Catalog (STAC) specification**

Current sensor resolutions and satellite revisit periods are such that terabytes of data products are added daily to the corresponding collections. Such datasets cannot be made accessible to users via full-catalog download. Space agencies and other data providers often offer access to their data catalogs through interactive Graphical User Interfaces (GUIs), see for instance the [Copernicus Open Access Hub portal](https://scihub.copernicus.eu/dhus/#/home) for the Sentinel missions. Accessing data via a GUI is a nice way to explore a catalog and get familiar with its content, but it represents a heavy and error-prone task that should be avoided if carried out systematically to retrieve data.

A service that offers programmatic access to the data enables users to reach the desired data in a more reliable, scalable and reproducible manner. An important element in the software interface exposed to the users, which is generally called the Application Programming Interface (API), is the use of standards. Standards, in fact, can significantly facilitate the reusability of tools and scripts across datasets and applications.

The SpatioTemporal Asset Catalog (STAC) specification is an emerging standard for describing geospatial data. By organizing metadata in a form that adheres to the STAC specifications, data providers make it possible for users to access data from different missions, instruments and collections using the same set of tools.


![Views of the STAC browser](https://carpentries-incubator.github.io/geospatial-python/fig/E05/STAC-browser.jpg)
Views of the radiant earth STAC browser

## More Resources on STAC
- [STAC specification](https://github.com/radiantearth/stac-spec#readme)
- [Tools based on STAC](https://stacindex.org/ecosystem)
- [STAC catalogs](https://stacindex.org/catalogs)

## Search a STAC catalog

The [STAC browser](https://radiantearth.github.io/stac-browser/#/) is a good starting point to discover available datasets, as it provides an up-to-date list of existing STAC catalogs. From the list, let's click on the "Earth Search" catalog, i.e. the access point to search the archive of Sentinel-2 images hosted on AWS.


## **Exercise:** Discover a STAC catalog
Let's take a moment to explore the Earth Search STAC catalog, which is the catalog indexing the Sentinel-2 collection
that is hosted on AWS. We can interactively browse this catalog using the STAC browser at [this link](https://radiantearth.github.io/stac-browser/#/external/earth-search.aws.element84.com/v1).

1. Open the link in your web browser. Which (sub-)catalogs are available?
2. Open the Sentinel-2 Level 2A collection, and select one item from the list. Each item corresponds to a satellite
"scene", i.e. a portion of the footage recorded by the satellite at a given time. Have a look at the metadata fields
and the list of assets. What kind of data do the assets represent?

In [ ]:
# Try something in here

## **Solution:**
(press three dots to reveal)


![Views of the Earth Search STAC endpoint](https://carpentries-incubator.github.io/geospatial-python/fig/E05/STAC-browser-exercise.jpg)

1. 7 subcatalogs are available, including a catalog for Landsat Collection 2, Level-2 and Sentinel-2 Level 2A (see left screenshot in the figure above).
2. When you select the Sentinel-2 Level 2A collection, and randomly choose one of the items from the list, you
should find yourself on a page similar to the right screenshot in the figure above. On the left side you will find
a list of the available assets: overview images (thumbnail and true color images), metadata files and the "real"
satellite images, one for each band captured by the Multispectral Instrument on board Sentinel-2.

When opening a catalog with the STAC browser, you can access the API URL by clicking on the "Source" button on the top
right of the page. By using this URL, we have access to the catalog content and, if supported by the catalog, to the
functionality of searching its items. For the Earth Search STAC catalog the API URL is:


In [ ]:
api_url = "https://earth-search.aws.element84.com/v1"

You can query a STAC API endpoint from Python using the `pystac_client` library:

In [ ]:
from pystac_client import Client

client = Client.open(api_url)


In the following, we ask for scenes belonging to the `sentinel-2-l2a` collection. This dataset includes Sentinel-2 data products pre-processed at level 2A (bottom-of-atmosphere reflectance) and saved in Cloud Optimized GeoTIFF (COG) format:

In [ ]:
collection = "sentinel-2-l2a"  # Sentinel-2, Level 2A, Cloud Optimized GeoTiffs (COGs)

---

## A note on cloud-optimized GeoTIFFs

Cloud Optimized GeoTIFFs (COGs) are regular GeoTIFF files with some additional features that make them ideal to be employed in the context of cloud computing and other web-based services. This format builds on the widely-employed GeoTIFF format, which you can find out more about in [Episode 1: Introduction to Raster Data](01-intro-raster-data.md). In short, a GeoTIFF is a standard .tif image format with additional spatial (georeferencing) information embedded in the file as tags. These tags should include the following raster metadata:
- Extent
- Resolution
- Coordinate Reference System (CRS)
- Values that represent missing data (NoDataValue)

COGs, by extension, are regular GeoTIFF files with a special internal structure. One of the features of COGs is that data is organized in "blocks" that can be accessed remotely via independent HTTP requests. Data users can thus access the only blocks of a GeoTIFF that are relevant for their analysis, without having to download the full file. In addition, COGs typically include multiple lower-resolution versions of the original image, called "overviews", which can also be accessed independently. By providing this "pyramidal" structure, users that are not interested in the details provided by a high-resolution raster can directly access the lower-resolution versions of the same image, significantly saving on the downloading time. More information on the COG format can be found [here](https://www.cogeo.org).

---

We also ask for scenes intersecting a geometry defined using the `shapely` library (in this case, a point):

In [ ]:
from shapely.geometry import Point
point = Point(4.89, 52.37)  # AMS coordinates

Note: at this stage, we are only dealing with metadata, so no image is going to be downloaded yet. But even metadata can be quite bulky if a large number of scenes match our search! For this reason, we limit the search result to 10 items:


In [ ]:
search = client.search(
    collections=[collection],
    intersects=point,
    max_items=10,
)

We submit the query and find out how many scenes match our search criteria (please note that this output can be different as more data is added to the catalog):



In [ ]:
print(search.matched())

Finally, we retrieve the metadata of the search results:

In [ ]:
items = search.item_collection()

The variable `items` is an `ItemCollection` object. We can check its size by:

In [ ]:
print(len(items))

which is consistent with the maximum number of items that we have set in the search criteria. We can iterate over the returned items and print these to show their IDs:

In [ ]:
for item in items:
    print(item)

Each of the items contains information about the scene geometry, its acquisition time, and other metadata that can be accessed as a dictionary from the `properties` attribute.

Let's inspect the metadata associated with the first item of the search results:

In [ ]:
item = items[0]
print(item.datetime)
print(item.geometry)
print(item.properties)

## **Exercise**: Search satellite scenes using metadata filters
Search for all the available Sentinel-2 scenes in the `sentinel-2-l2a` collection that satisfy the following criteria:
- intersect a provided bounding box (use ±0.01 deg in lat/lon from the previously defined point);
- have been recorded between 20 March 2020 and 30 March 2020;
- have a cloud coverage smaller than 10% (hint: use the `query` input argument of `client.search`).

How many scenes are available? Save the search results in GeoJSON format as `search.json`.

In [ ]:
# Try something in here:

## **Solution**:
(press three dots to reveal)

In [ ]:
bbox = point.buffer(0.01).bounds

In [ ]:
search = client.search(
    collections=[collection],
    bbox=bbox,
    datetime="2020-03-20/2020-03-30",
    query=["eo:cloud_cover<15"]
)
print(search.matched())

In [ ]:
items = search.item_collection()
items.save_object("search.json")

## Access the assets

So far we have only discussed metadata - but how can one get to the actual images of a satellite scene (the "assets" in the STAC nomenclature)? These can be reached via links that are made available through the item's attribute `assets`.


In [ ]:
assets = items[0].assets  # first item's asset dictionary
print(assets.keys())


We can print a minimal description of the available assets:

In [ ]:
for key, asset in assets.items():
    print(f"{key}: {asset.title}")

Among the others, assets include multiple raster data files (one per optical band, as acquired by the multi-spectral instrument), a thumbnail, a true-color image ("visual"), instrument metadata and scene-classification information ("SCL"). Let's get the URL links to the actual asset:


In [ ]:
print(assets["thumbnail"].href)

This can be used to download the corresponding file:

![Overview of the true-colour image](https://carpentries-incubator.github.io/geospatial-python/fig/E05/STAC-s2-preview.jpg)

###### Overview of the true-colour image ("thumbnail")


Remote raster data can be directly opened via the `rioxarray` library. We will
learn more about this library in the next part of the notebook.

In [ ]:
import rioxarray
nir_href = assets["nir"].href
nir = rioxarray.open_rasterio(nir_href)
print(nir)


We can then save the data to disk:

In [ ]:
# save whole image to disk
# NOTE: This might take a while
nir.rio.to_raster("nir.tif")

Since that might take a while, given there are over 10000 x 10000 = a hundred million pixels in the 10 meter NIR band, you can take a smaller subset before downloading it. Becuase the raster is a COG, we can download just what we need!

Here, we specify that we want to download the first (and only) band in the tif file, and a slice of the width and height dimensions.


In [ ]:
# save portion of an image to disk
nir[0,1500:2200,1500:2200].rio.to_raster("nir_subset.tif")

The difference is 155 Megabytes for the large image vs about 1 Megabyte for the subset.


## **Exercise:** Downloading Landsat 8 Assets
In this exercise we put in practice all the skills we have learned thusfar to retrieve images from a different mission: [Landsat 8](https://www.usgs.gov/landsat-missions/landsat-8). In particular, we browse images from the [Harmonized Landsat Sentinel-2 (HLS) project](https://lpdaac.usgs.gov/products/hlsl30v002/), which provides images from NASA's Landsat 8 and ESA's Sentinel-2 that have been made consistent with each other. The HLS catalog is indexed in the NASA Common Metadata Repository (CMR) and it can be accessed from the STAC API endpoint at the following URL:
`https://cmr.earthdata.nasa.gov/stac/LPCLOUD`.

1. Using `pystac_client`, search for all assets of the Landsat 8 collection (`HLSL30.v2.0`) from February to March
  2021, intersecting the point with longitude/latitute coordinates (-73.97, 40.78) deg.
2. Visualize an item's thumbnail (asset key `browse`).

In [ ]:
# Try something in here

## **Solution:**
(click on each of the three dots to expand each answer)

In [ ]:
# connect to the STAC endpoint
cmr_api_url = "https://cmr.earthdata.nasa.gov/stac/LPCLOUD"
client = Client.open(cmr_api_url)

# setup search
search = client.search(
    collections=["HLSL30.v2.0"],
    intersects=Point(-73.97, 40.78),
    datetime="2021-02-01/2021-03-30",
) # nasa cmr cloud cover filtering is currently broken: https://github.com/nasa/cmr-stac/issues/239

# retrieve search results
items = search.item_collection()
print(len(items))

In [ ]:
items_sorted = sorted(items, key=lambda x: x.properties["eo:cloud_cover"]) # sorting and then selecting by cloud cover
item = items_sorted[0]
print(item)

In [ ]:
print(item.assets["browse"].href)

![Thumbnail of the Landsat-8 scene](https://d1nklfio7vscoe.cloudfront.net/s3-2d2df3a34830d5223d1e9547cd713408/lp-prod-public.s3.us-west-2.amazonaws.com/HLSL30.020/HLS.L30.T18TWL.2021039T153324.v2.0/HLS.L30.T18TWL.2021039T153324.v2.0.jpg?A-userid=None&Expires=1699441799&Signature=eqm-82ZSV1O15ydTG2TYuyRSD7ys2IZXOZd3GGDpYyzoBvY8gnLoaNihvJrQcw04ly-KgbPPWIiffN2zu-CTCmNb4~HUii1kxxyLUkBYwtKld3KmAqpnRZrA1AiqZG4JppWO3Hi7g96TJbxfuavMsul9YGifD6zTW-BIpCU-WhN8rv68TtwlhYORvD8u6TD4oAAtDroXUNJe-u3V2sNTNrGPC8l-7fKb7akGuN78DZUF~wsCVwkyTmQPsCahgxlvU8V4b6yrq5iYA7fypsfiQRx-IciqrJdcTxC1Zvncs7~3GLgOME86d2z1gxW8SKsSe33cIdXpmwsM40c3KJ8RFA__&Key-Pair-Id=K24EGHOUJPFCBQ)

Thumbnail of the Landsat-8 scene

## Public catalogs, protected data

Publicly accessible catalogs and STAC endpoints do not necessarily imply publicly accessible data. Data providers, in
fact, may limit data access to specific infrastructures and/or require authentication. For instance, the NASA CMR STAC
endpoint considered in the last exercise offers publicly accessible metadata for the HLS collection, but most of the
linked assets are available only for registered users (the thumbnail is publicly accessible).

The authentication procedure for dataset with restricted access might differ depending on the data provider. For the
NASA CMR, follow these steps in order to access data using Python:

* Create a NASA Earthdata login account [here](https://urs.earthdata.nasa.gov);
* Set up a netrc file with your credentials, e.g. by using [this script](https://git.earthdata.nasa.gov/projects/LPDUR/repos/daac_data_download_python/browse/EarthdataLoginSetup.py);
* Define the following environment variables:



In [ ]:
import os
os.environ["GDAL_HTTP_COOKIEFILE"] = "./cookies.txt"
os.environ["GDAL_HTTP_COOKIEJAR"] = "./cookies.txt"

## Key takeaways:
Accessing satellite images via the providers' API enables a more reliable and scalable data retrieval.
 - STAC catalogs can be browsed and searched using the same tools and scripts.
 - `rioxarray` allows you to open and download remote raster files.
 
---

# 2. Read and visualise raster data

Next, we introduce the fundamental principles, packages and metadata/raster attributes for working with raster data in Python. We will also explore how Python handles missing and bad data values.

[`rioxarray`](https://corteva.github.io/rioxarray/stable/) is the Python package we will use throughout the rest of this notebook to work with raster data. It is based on the popular [`rasterio`](https://rasterio.readthedocs.io/en/latest/) package for working with rasters and [`xarray`](https://xarray.pydata.org/en/stable/) for working with multi-dimensional arrays.
`rioxarray` extends `xarray` by providing top-level functions (e.g. the `open_rasterio` function to open raster datasets) and by adding a set of methods to the main objects of the `xarray` package (the `Dataset` and the `DataArray`). These additional methods are made available via the `rio` accessor and become available from `xarray` objects after importing `rioxarray`.

We will also use the [`pystac`](https://github.com/stac-utils/pystac) package to load rasters from the search results we created in the previous section.

### About Raster Data

Raster data is any pixelated (or gridded) data where each pixel is associated
with a specific geographic location. The value of a pixel can be
continuous (e.g. elevation) or categorical (e.g. land use). If this sounds
familiar, it is because this data structure is very common: it's how
we represent any digital image. A geospatial raster is only different
from a digital photo in that it is accompanied by spatial information
that connects the data to a particular location. This includes the
raster's extent and cell size, the number of rows and columns, and
its coordinate reference system (or CRS).

![raster-concept](https://carpentries-incubator.github.io/geospatial-python/fig/E01/raster_concept.png)
###### Raster Concept (Source: National Ecological Observatory Network (NEON))

Some examples of continuous rasters include:

1. Precipitation maps.
2. Maps of tree height derived from LiDAR data.
3. Elevation values for a region.

A map of elevation for Harvard Forest derived from the [NEON AOP LiDAR sensor](https://www.neonscience.org/data-collection/airborne-remote-sensing)
is below. Elevation is represented as a continuous numeric variable in this map. The legend
shows the continuous range of values in the data from around 300 to 420 meters.

![elevation plot](https://carpentries-incubator.github.io/geospatial-python/fig/E01/continuous-elevation-HARV-plot-01.png)
###### Continuous Elevation Map: HARV Field Site

For more information and further examples of raster data you can visit the [relevant lesson](01-intro-raster-data.md) in the software carpentry course this notebook is based off. 


## Load a Raster and View Attributes
In the previous episode, we searched for Sentinel-2 images, and then saved the search results to a file: `search.json`. This contains the information on where and how to access the target images from a remote repository. We can use the function `pystac.ItemCollection.from_file()` to load the search results as an `Item` list.


In [ ]:
items = pystac.ItemCollection.from_file("search.json")
items

In the search results, we have 6 `Item` type objects, corresponding to several Sentinel-2 scenes from March 21th and 28th in 2020. We will focus on the scene `S2A_31UFU_20200328_0_L2A`, and load band `nir09` (central wavelength 945 nm). We can load this band using the function `rioxarray.open_rasterio()`, via the Hypertext Reference `href` (commonly referred to as a URL):

## **Exercise:** finding the right item and asset
How do we go about selecting the correct item and asset from our ItemCollection we just loaded?
1. Find the item corresponding to scene S2A_31UFU_20200328_0_L2A
2. Find the asset `href` for the `nir09` band
3. Load it using rioxarray's `open_rasterio` method into a variable called `raster_ams_b9`

In [ ]:
# Try something in here

## **Solution**:
(press on each of the three dots to reveal)

In [ ]:
for item in items:
    if item.id == "S2A_31UFU_20200328_0_L2A":
        break
item

In [ ]:
item.assets['nir09'].href

In [ ]:
raster_ams_b9 = rioxarray.open_rasterio(item.assets["nir09"].href)

By calling the variable name in the jupyter notebook we can get a quick look at the shape and attributes of the data.

In [ ]:
raster_ams_b9

The first call to `rioxarray.open_rasterio()` opens the file from remote or local storage, and then returns a `xarray.DataArray` object. The object is stored in a variable, i.e. `raster_ams_b9`. Reading in the data with `xarray` instead of `rioxarray` also returns a `xarray.DataArray`, but the output will not contain the geospatial metadata (such as projection information). You can use numpy functions or built-in Python math operators on a `xarray.DataArray` just like a numpy array. Calling the variable name of the `DataArray` also prints out all of its metadata information.

The output tells us that we are looking at an `xarray.DataArray`, with `1` band, `1830` rows, and `1830` columns. We can also see the number of pixel values in the `DataArray`, and the type of those pixel values, which is unsigned integer (or `uint16`). The `DataArray` also stores different values for the coordinates of the `DataArray`. When using `rioxarray`, the term coordinates refers to spatial coordinates like `x` and `y` but also the `band` coordinate. Each of these sequences of values has its own data type, like `float64` for the spatial coordinates and `int64` for the `band` coordinate.

This `DataArray` object also has a couple of attributes that are accessed like `.rio.crs`, `.rio.nodata`, and `.rio.bounds()`, which contain the metadata for the file we opened. Note that many of the metadata are accessed as attributes without `()`, but `bounds()` is a method (i.e. a function in an object) and needs parentheses.


In [ ]:
print(raster_ams_b9.rio.crs)
print(raster_ams_b9.rio.nodata)
print(raster_ams_b9.rio.bounds())
print(raster_ams_b9.rio.width)
print(raster_ams_b9.rio.height)

The Coordinate Reference System, or `raster_ams_b9.rio.crs`, is reported as the string `EPSG:32631`. The `nodata` value is encoded as 0 and the bounding box corners of our raster are represented by the output of `.bounds()` as a `tuple` (like a list but you can't edit it). The height and width match what we saw when we printed the `DataArray`, but by using `.rio.width` and `.rio.height` we can access these values if we need them in calculations.

We will be exploring this data throughout this episode. By the end of this episode, you will be able to understand and explain the metadata output.


<span style="color: blue;">*TIP: To improve code readability, file and object names should be used that make it clear what is in the file. The data for this episode covers Amsterdam, and is from Band 9, so we'll use a naming convention of `raster_ams_b9` for the variable name.*</span>


## Visualize a Raster

After viewing the attributes of our raster, we can examine the raw values of the array with `.values`:




In [ ]:
raster_ams_b9.values

This can give us a quick view of the values of our array, but only at the corners. Since our raster is loaded in python as a `DataArray` type, we can plot this in one line similar to a pandas `DataFrame` with `DataArray.plot()`.

In [ ]:
raster_ams_b9.plot()

Nice plot! Notice that `rioxarray` helpfully allows us to plot this raster with spatial coordinates on the x and y axis (this is not the default in many cases with other functions or libraries).

This plot shows the satellite measurement of the spectral band `nir09` for an area that covers part of the Netherlands. According to the [Sentinel-2 documentaion](https://sentinels.copernicus.eu/web/sentinel/technical-guides/sentinel-2-msi/msi-instrument), this is a band with the central wavelength of 945nm, which is sensitive to water vapor. It has a spatial resolution of 60m. Note that the `band=1` in the image title refers to the ordering of all the bands in the  `DataArray`, not the Sentinel-2 band number `09` that we saw in the pystac search results.

With a quick view of the image, we notice that half of the image is blank, no data is captured. We also see that the cloudy pixels at the top have high reflectance values, while the contrast of everything else is quite low. This is expected because this band is sensitive to the water vapor. However if one would like to have a better color contrast, one can add the option `robust=True`, which displays values between the 2nd and 98th percentile:

In [ ]:
raster_ams_b9.plot(robust=True)

Now the color limit is set in a way fitting most of the values in the image. We have a better view of the ground pixels.

---

*NOTE: The option `robust=True` always forces displaying values between the 2nd and 98th percentile. Of course, this will not work for every case. For a customized displaying range, you can also manually specifying the keywords `vmin` and `vmax`. For example ploting between `100` and `7000`:*

In [ ]:
raster_ams_b9.plot(vmin=100, vmax=7000)

---

## View Raster Coordinate Reference System (CRS) in Python
Another information that we're interested in is the CRS, and it can be accessed with `.rio.crs`. We introduced the concept of a CRS in [an earlier
episode](03-crs.md).
Now we will see how features of the CRS appear in our data file and what
meanings they have. We can view the CRS string associated with our DataArray's `rio` object using the `crs`
attribute.


In [ ]:
print(raster_ams_b9.rio.crs)


To print the EPSG code number as an `int`, we use the `.to_epsg()` method:

In [ ]:
raster_ams_b9.rio.crs.to_epsg()

EPSG codes are great for succinctly representing a particular coordinate reference system. But what if we want to see more details about the CRS, like the units? For that, we can use `pyproj`, a library for representing and working with coordinate reference systems.

In [ ]:
from pyproj import CRS
epsg = raster_ams_b9.rio.crs.to_epsg()
crs = CRS(epsg)
crs

The `CRS` class from the `pyproj` library allows us to create a `CRS` object with methods and attributes for accessing specific information about a CRS, or the detailed summary shown above.

A particularly useful attribute is `area_of_use`, which shows the geographic bounds that the CRS is intended to be used.



In [ ]:
crs.area_of_use

## **Exercise**: find the axes units of the CRS
What units are our data in? See if you can find a method to examine this information using `help(crs)` or `dir(crs)`

In [ ]:
# Try something in here

## **Solution**:
(press three dots to reveal)

`crs.axis_info` tells us that the CRS for our raster has two axes and both are in meters.
We could also get this information from the attribute `raster_ams_b9.rio.crs.linear_units`.

Let's break down the pieces of the `pyproj` CRS summary. The string contains all of the individual CRS elements that Python or another GIS might need, separated into distinct sections, and datum.

In [ ]:
crs

* **Name** of the projection is UTM zone 31N (UTM has 60 zones, each 6-degrees of longitude in width). The underlying datum is WGS84.
* **Axis Info**: the CRS shows a Cartesian system with two axes, easting and northing, in meter units.
* **Area of Use**: the projection is used for a particular range of longitudes `0°E to 6°E` in the northern hemisphere (`0.0°N to 84.0°N`)
* **Coordinate Operation**: the operation to project the coordinates (if it is projected) onto a cartesian (x, y) plane. Transverse Mercator is accurate for areas with longitudinal widths of a few degrees, hence the distinct UTM zones.
* **Datum**: Details about the datum, or the reference point for coordinates. `WGS 84` and `NAD 1983` are common datums. `NAD 1983` is [set to be replaced in 2022](https://en.wikipedia.org/wiki/Datum_of_2022).

Note that the zone is unique to the UTM projection. Not all CRSs will have a
zone. Below is a simplified view of US UTM zones.

![UTMZones](https://upload.wikimedia.org/wikipedia/commons/thumb/8/8d/Utm-zones-USA.svg/1920px-Utm-zones-USA.svg.png)
###### The UTM zones across the continental United States (Chrismurf at English Wikipedia, via [Wikimedia Commons](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system#/media/File:Utm-zones-USA.svg) (CC-BY))

## Calculate Raster Statistics

It is useful to know the minimum or maximum values of a raster dataset. We can compute these and other descriptive statistics with `min`, `max`, `mean`, and `std`.


In [ ]:
print(raster_ams_b9.min())
print(raster_ams_b9.max())
print(raster_ams_b9.mean())
print(raster_ams_b9.std())

The information above includes a report of the min, max, mean, and standard deviation values, along with the data type. If we want to see specific quantiles, we can use xarray's `.quantile()` method. For example for the 25% and 75% quantiles:

In [ ]:
print(raster_ams_b9.quantile([0.25, 0.75]))

---
*NOTE: You could also get each of these values one by one using `numpy`.*


In [ ]:
import numpy
print(numpy.percentile(raster_ams_b9, 25))
print(numpy.percentile(raster_ams_b9, 75))

You may notice that `raster_ams_b9.quantile` and `numpy.percentile` didn't require an argument specifying the axis or dimension along which to compute the quantile. This is because `axis=None` is the default for most numpy functions, and therefore `dim=None` is the default for most xarray methods. It's always good to check out the docs on a function to see what the default arguments are, particularly when working with multi-dimensional image data. To do so, we can use`help(raster_ams_b9.quantile)` (or `?raster_ams_b9.percentile` in jupyter notebook), e.g.


In [ ]:
?raster_ams_b9.quantile

---

## Dealing with Missing Data
So far, we have visualized a band of a Sentinel-2 scene and calculated its statistics. However, we need to take missing data into account. Raster data often has a "no data value" associated with it and for raster datasets read in by `rioxarray`. This value is referred to as `nodata`. This is a value assigned to pixels where data is missing or no data were collected. There can be different cases that cause missing data, and it's common for other values in a raster to represent different cases. The most common example is missing data at the edges of rasters.

By default the shape of a raster is always rectangular. So if we have a dataset that has a shape that isn't rectangular, some pixels at the edge of the raster will have no data values. This often happens when the data were collected by a sensor which only flew over some part of a defined region.

As we have seen above, the `nodata` value of this dataset (`raster_ams_b9.rio.nodata`) is 0. When we have plotted the band data, or calculated statistics, the missing value was not distinguished from other values. Missing data may cause some unexpected results. For example, the 25th percentile we just calculated was 0, probably reflecting the presence of a lot of missing data in the raster.

To distinguish missing data from real data, one possible way is to use `nan` to represent them. This can be done by specifying `masked=True` when loading the raster:


In [ ]:
raster_ams_b9 = rioxarray.open_rasterio(items[0].assets["nir09"].href, masked=True)

One can also use the `where` function to select all the pixels which are different from the `nodata` value of the raster:

In [ ]:
raster_ams_b9.where(raster_ams_b9!=raster_ams_b9.rio.nodata)

Either way will change the `nodata` value from 0 to `nan`. Now if we compute the statistics again, the missing data will not be considered:

In [ ]:
print(raster_ams_b9.min())
print(raster_ams_b9.max())
print(raster_ams_b9.mean())
print(raster_ams_b9.std())

And if we plot the image, the `nodata` pixels are not shown because they are not 0 anymore:

In [ ]:
raster_ams_b9.plot(robust=True)

One should notice that there is a side effect of using `nan` instead of `0` to represent the missing data: the data type of the `DataArray` was changed from integers to float. This need to be taken into consideration when the data type matters in your application.

## Raster Bands
So far we looked into a single band raster, i.e. the `nir09` band of a Sentinel-2 scene. However, to get a smaller, non georeferenced version of the scene, one may also want to visualize the true-color overview of the region. This is provided as a multi-band raster -- a raster dataset that contains more than one band.

![Sketch of a multi-band raster image](https://carpentries-incubator.github.io/geospatial-python/fig/E06/single_multi_raster.png)
###### Sketch of a multi-band raster image

The `overview` asset in the Sentinel-2 scene is a multiband asset. Similar to `nir09`, we can load it by:

In [ ]:
raster_ams_overview = rioxarray.open_rasterio(items[0].assets['visual'].href, overview_level=3)
raster_ams_overview


The band number comes first when GeoTiffs are read with the `.open_rasterio()` function. As we can see in the `xarray.DataArray` object, the shape is now `(band: 3, y: 687, x: 687)`, with three bands in the `band` dimension. It's always a good idea to examine the shape of the raster array you are working with and make sure it's what you expect. Many functions, especially the ones that plot images, expect a raster array to have a particular shape. One can also check the shape using the `.shape` attribute:

In [ ]:
raster_ams_overview.shape

One can visualize the multi-band data with the `DataArray.plot.imshow()` function:

In [ ]:
raster_ams_overview.plot.imshow()

Note that the `DataArray.plot.imshow()` function makes assumptions about the shape of the input DataArray, that since it has three channels, the correct colormap for these channels is RGB. It does not work directly on image arrays with more than 3 channels. One can replace one of the RGB channels with another band, to make a false-color image.

## **Exercise**: set the plotting aspect ratio
As seen in the figure above, the true-color image is stretched. Visualize it with the right aspect ratio. You can use the [documentation](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.plot.imshow.html) of `DataArray.plot.imshow()`.
Since we know the height/width ratio is 1:1 (check the `rio.height` and `rio.width` attributes), we can set the aspect ratio to be 1. For example, we can choose the size to be 5 inches, and set `aspect=1`. Note that according to the [documentation](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.plot.imshow.html) of `DataArray.plot.imshow()`, when specifying the `aspect` argument, `size` also needs to be provided.

In [ ]:
# Try something in here

## **Solution**:
(press each of the three dots to reveal)

We can calculate the aspect ratio with the `rio.height` and `rio.width` properties on our rioxarray dataset. Remember we need to use the `.rio` accessor to access rasterio's properties (see [the xarray docs](https://docs.xarray.dev/en/stable/internals/extending-xarray.html) for more info if you're interested).

In [ ]:
# Check the aspect ratio
h = raster_ams_overview.rio.height
w = raster_ams_overview.rio.width
print(f"Height: {h}, Width: {w}")
aspect_ratio = h/w
print(f"Aspect ratio: {aspect_ratio}")

We can then set the kwarg `aspect=` to our calculated value for aspect ratio. Note that according to the [documentation](https://xarray.pydata.org/en/stable/generated/xarray.DataArray.plot.imshow.html) of `DataArray.plot.imshow()`, when specifying the `aspect` argument, `size` also needs to be provided, so we just choose the size to be 5 inches.

In [ ]:
raster_ams_overview.plot.imshow(size=5, aspect=aspect_ratio)

## Key takeaways:
- `rioxarray` and `xarray` are for working with multidimensional arrays like pandas is for working with tabular data.
- `rioxarray` stores CRS information as a CRS object that can be converted to an EPSG code or PROJ4 string.
- Missing raster data are filled with nodata values, which should be handled with care for statistics and visualization.